In [ ]:
import os
os.chdir('..')
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")

from weak_learned import Pipeline
from collections import namedtuple
from animalai.envs.arena_config import ArenaConfig
margs = namedtuple('args', 'env seed arenas num_episodes inference distribution max_steps mode save_path')
env_path = '../env/aaiagain'
arenas = [
#     "configurations/curriculum4/35.yml",
#     "../competition_configurations/4-1-2.yml", #red
#     "../competition_configurations/3-16-1.yml",
#     "../competition_configurations/3-16-2.yml",
#     "../competition_configurations/3-16-3.yml",
#     "../competition_configurations/3-18-1.yml",
#     "../competition_configurations/3-18-2.yml",
#     "../competition_configurations/3-18-3.yml",
#     "../competition_configurations/3-19-1.yml",
#     "../competition_configurations/3-19-2.yml",
#     "../competition_configurations/3-19-3.yml",
#     "training_set/ramp.yml",
#     "training_set/ramp.yml",
#     "training_set/wall.yml"
#     "training_set/choice.yml",
#     "training_set/numerosity.yml",
    "training_set/ymaze2.yml"
#     "../competition_configurations/8-19-2.yml",

]
distribution = [1]
max_steps = {
    "training_set/ramp.yml":2,
    "training_set/red_maze.yml":1,
    "training_set/wall.yml":3,
    "training_set/choice.yml":2,
    "training_set/numerosity.yml":1,
    "training_set/moving.yml":5,
    "training_set/ymaze2.yml":0,
    "../competition_configurations/8-19-2.yml":10,

}

# seed = 1
# ac = [ArenaConfig(f"../competition_configurations/{i}.yml") for i in x]
# print(f"Total number of arenas:{len(ac)}")
num = 4
test = True
args = margs(env=env_path, seed=2, arenas=arenas, num_episodes=num+1,
             inference=True, distribution=distribution,
            max_steps=max_steps, mode='collect',save_path=f"biased_traces/test.txt")
pipe = Pipeline(args, test)
pipe.buffer_size = num
res = pipe.learn_run()


In [ ]:
x = None
bool(x)

8/10 success
Dont rotate if something is visible
Interacting with anything is penalised
Exploring is good when exploring the goal WRONG
:~ initiate(interact(V1)).[1@2, V1]
:~ not initiate(rotate), visible(V1).[-1@3, V1]
:~ goal(V1), initiate(explore(V1,V2)).[-1@1, V1, V2]
    
Success num: 14/20
Explore whatever is occluded and if nothing is occluded
interact with goal
NEW RULES LEARNED: 
:~ goal(V1), not initiate(interact(V1)).[1@1, V1]
:~ not initiate(explore(V1,V2)), occludes(V1,V2).[1@3, V1, V2]

Success num: 22/30
If goal is visible interact with it
Otherwise explore the goal WRONG
Otherwsie rotate
NEW RULES LEARNED: 
:~ initiate(rotate).[-1@1]
:~ goal(V1), initiate(explore(V1,V2)).[-1@2, V1, V2]
:~ goal(V1), initiate(interact(V1)), visible(V1).[-1@3, V1]

Noticing that it converges quite fast to policy. Need to add random noise
with decaying epsilon to ensure it doesnt


Success num: 33/40
If something is occluding, explore it
Otherwise interact with goal
NEW RULES LEARNED: 
:~ not initiate(explore(V1,V2)), occludes(V1,V2).[1@3, V1, V2]
:~ goal(V1), not initiate(interact(V1)).[1@2, V1]

Optimal convergence 47/50
If goal is visible interact, if anything is visible explore
otherwise interacting with something is penalised so better to rotate
NEW RULES LEARNED: 
:~ initiate(interact(V1)).[1@1, V1]
:~ initiate(explore(V1,V2)), visible(V1).[-1@2, V1, V2]
:~ goal(V1), initiate(interact(V1)), visible(V1).[-1@3, V1]

In [ ]:
NEW RULES LEARNED: :~ initiate(interact(V1)).[1@1, V1]
:~ initiate(explore(V1,V2)), visible(V1).[-1@2, V1, V2]
:~ not initiate(interact(V1)), not wall(V1), visible(V1).[1@3, V1]
:~ goal(V2), initiate(avoid(V2,V3)), lava(V1).[-1@4, V1, V2, V3]
    

:~ initiate(avoid(V1,V2)), not goal(V1), visible(V1).[-1@1, V1, V2]
:~ initiate(explore(V1,V2)).[-1@2, V1, V2]
:~ initiate(explore(V1,V2)), visible(V1).[1@3, V1, V2]
:~ initiate(interact(V1)).[1@4, V1]


VERY INTERESTING:
Looks like the avoid macro-action is better than interact even when goal is visible
#pos(a2,
{},
{},
{visible(a).
visible(b).
wall(b).
goal(a).
initiate(avoid(b,a)).}).
%%Value was:6.363636363636363
#pos(a3,
{},
{},
{visible(a).
visible(b).
wall(b).
goal(a).
initiate(interact(a)).}).
%%Value was:5.2375

    
    

In [ ]:
import time
results = {}
for i in range(10):
    start = time.time()
    margs = namedtuple('args', 'seed env arena_config num_episodes inference')
    args = margs(env=env_path, seed =i, arena_config=ac, num_episodes=90, inference=False)
    pipe = Pipeline(args)
    results[i] = pipe.learn_run()
    end = time.time()


In [ ]:
from clyngor import ASP
lp = """
adjacent(12,20).
adjacent(20,12).
on(agent,20).
visible(10, 11).
wall(10).
visible(11, 54).
wall(11).
visible(12, 44).
wall(12).
visible(20, 452).
platform(20).
goal(42).
present(X):-goal(X).
present(X):- visible(X,_).
separator(Y):-on(agent, X), adjacent(X, Y), platform(X).
can_occlude(X):-wall(X), not separator(X).
occludes(X,Y, O) :- present(Y), visible(X, O), not visible(Y, _), can_occlude(X).
occludes_more(X, Y) :- occludes(X,Z,O1), occludes(Y,Z,O2), O1 > O2."""

In [ ]:
p = next(ASP(lp).sorted)
lp = ".\n".join(next(ASP(lp).atoms_as_string)) + '.'


In [1]:
import os
os.chdir('..')

In [20]:
from collections import deque, namedtuple
from clyngor import ASP
import operator
ctx_observables = [
    'on',
    'occludes',
    'occludes_more',
    'bigger',
    'more_goals',
    'moving',
    'wall',
    'goal',
    'platform',
    'lava',
    'ramp',
    'goal1',

    'rotate',
    'observe',
    'drop',
    'interact',
    'climb',
    'explore',
    'balance',
    'avoid',
    'collect'
    ]
arenas = ['ymaze', 'choice', 'numerosity','red_maze', 'wall']
arenas = ['ramp']

# Concat traces
traces = []
for arena in arenas:
    my_file = open(f"early_traces/{arena}.txt", "r")
    content = eval(my_file.read())
    s = len([i for i in content if i[2]])
    t = len(content)
    l = t-s
    reward = [-s/t*10, l/t*20] # fail, succ
    for c,i in enumerate(content):
        content[c][2] = reward[content[c][2]]
        if arena!='moving':
            content[c][1][0] = content[c][1][0].replace('moving.\n', '')
    traces+= content
AnswerSet = namedtuple('AS', ['r', 'p', 'a', 'o']) # r for raw, p for parsed, a for arity
parse_single_args = lambda x: list(list(ASP(x+'.').parse_args)[0])[0]
# parse_args = lambda x: list(list(ASP(x).sorted)[0])
main_lp = """
separator(Y):-on(agent, platform), adjacent(X, Y).
can_occlude(X):-wall(X), not separator(X).
gvis:-goal(X).
occluding(X, O) :- not gvis, can_occlude(X), visible(X,O).
occludes(X):-occluding(X,_).
occludes_more(X, Y) :- occluding(X,O1), occluding(Y,O2), O1 > O2.
bigger(X,Y):- goal(X), goal(Y), visible(X,O1), visible(Y,O2), O1>O2.
"""
# def parse_args(x):
#     x = list(ASP(x).sorted)
#     return list(x[0])
def extract_action(action):
    res = action.split('initiate(')
    res = res[0].split('(')[0]
    return res
def expand_trace(trace):
    discount_factor = 0.9
    actions, states, success, len_trace = trace
    values = []
    for step in range(len_trace):
        values.append(success*discount_factor**(len_trace-step-1))
    return actions, states, values
def flatten_macros(p):
    res = []
    for i in p:
        if i[1]:
            if isinstance(i[1][0],  tuple):
                res.append(i[1][0])
            else:
                res.append(i)
        else:
            res.append(i)
    return res
def variabilise(lp):
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm','n','o','p','q','r','s','t','u','v','w','x','y','z']
    lp = lp.replace('left','1000001').replace('right', '100002')
    lp = ".\n".join(i for i in next(ASP(lp+main_lp).atoms_as_string.sorted) if any(j in i for j in ctx_observables)&(not any(j in i for j in ['ramp(', 'lava(','platform(', 'goal1('])))
    if lp:
        lp+= '.'
        p = next(ASP(lp).parse_args.sorted)
        y = flatten_macros(p)
        # Create unique var map
        var_map = {}
        for lit in y:
            for arg in lit[1]:
                if (arg not in var_map):
                    if isinstance(arg, int):
                        var_map[arg] = letters.pop(0)

        # Sort vars by largest so smaller ones aren't replacing bigger numbers
        order = sorted(var_map, reverse=True)
        # Update lp
        for var in order:
            lp = lp.replace(str(var), var_map[var])
    return lp

In [21]:
actions = []
states = []
values = []
for trace in traces:
    a, s, v = expand_trace(trace)
    actions += a
    states += s
    values += v
pairs = [variabilise(s+f"{a}.") for s,a in zip(states,actions)]
unique_pairs = list(set(pairs))
var_states = [variabilise(s) for s in states]
unique_states = list(set(var_states))
pairs = [[unique_pairs.index(p),unique_states.index(s)] for s,p in zip(var_states, pairs)]

tree = {s: {} for s in range(len(unique_states))}
for c, p in enumerate(pairs):
    if p[0] in tree[p[1]]:
        tree[p[1]][p[0]].append(c)
    else:
        tree[p[1]][p[0]] = [c]

examples = ""
order = ""
e_c = 0
o_c = 0
# Rolling average for each action
for s,d in tree.items():
    if len(d)==1:
        continue
    for a,v in d.items():
        tree[s][a] = sum(values[i] for i in v)/len(v)

    ranking = sorted(tree[s].items(), key=operator.itemgetter(1), reverse=True)
    top_action = e_c
    if ranking[0][1]<0: # Don't add examples or ordering when the best action didn't lead to positive episodes
        continue
    for c, i in enumerate(ranking):
        action, val = i
        examples += f"#pos(a{e_c},\n{{}},\n{{}},\n{{{unique_pairs[action]}}}).\n%%Value was:{val}\n"
        if c!=0:
            order+= f"#brave_ordering(b{o_c}@10, a{top_action}, a{e_c}).\n"
            o_c +=1
        e_c +=1

In [22]:
unique_states

['goal(a).\nplatform.',
 'on(agent,platform).\nplatform.',
 'goal(a).\non(goal,platform).\nplatform.',
 'goal(a).\non(goal,platform).\nplatform.\nramp.',
 'goal(a).\non(agent,platform).\non(goal,platform).\nplatform.',
 'platform.']

In [23]:
unique_pairs

['goal(a).\ninitiate(balance).\non(agent,platform).\non(goal,platform).\nplatform.',
 'goal(a).\ninitiate(observe).\non(agent,platform).\non(goal,platform).\nplatform.',
 'goal(a).\ninitiate(observe).\non(goal,platform).\nplatform.',
 'initiate(drop(a)).\non(agent,platform).\nplatform.',
 'goal(a).\ninitiate(interact(a)).\non(goal,platform).\nplatform.',
 'goal(a).\ninitiate(interact(a)).\non(agent,platform).\non(goal,platform).\nplatform.',
 'goal(a).\ninitiate(rotate).\non(agent,platform).\non(goal,platform).\nplatform.',
 'initiate(rotate).\non(agent,platform).\nplatform.',
 'goal(a).\ninitiate(observe).\nplatform.',
 'goal(a).\ninitiate(interact(a)).\non(goal,platform).\nplatform.\nramp.',
 'goal(a).\ninitiate(drop(b)).\non(agent,platform).\non(goal,platform).\nplatform.',
 'initiate(balance).\non(agent,platform).\nplatform.',
 'goal(a).\ninitiate(rotate).\non(goal,platform).\nplatform.',
 'goal(a).\ninitiate(rotate).\non(goal,platform).\nplatform.\nramp.',
 'initiate(rotate).\npla

In [24]:
tree

{0: {8: [603]},
 1: {3: -0.6578947368421052, 11: 18.68421052631579, 7: -0.6578947368421052},
 2: {12: -0.6578947368421051, 4: -0.657894736842105, 2: -0.6578947368421049},
 3: {9: -0.6113883847549909,
  16: 2.215447154471546,
  17: -0.6122076023391817,
  13: -0.6133705475810743},
 4: {5: -0.6578947368421052,
  1: -0.6578947368421052,
  6: -0.6578947368421052,
  0: 15.921052631578945,
  10: -0.6578947368421052},
 5: {15: -0.6578947368421052, 14: -0.6578947368421052}}

In [25]:
print(examples)

#pos(a0,
{},
{},
{initiate(balance).
on(agent,platform).
platform.}).
%%Value was:18.68421052631579
#pos(a1,
{},
{},
{initiate(drop(a)).
on(agent,platform).
platform.}).
%%Value was:-0.6578947368421052
#pos(a2,
{},
{},
{initiate(rotate).
on(agent,platform).
platform.}).
%%Value was:-0.6578947368421052
#pos(a3,
{},
{},
{goal(a).
initiate(climb).
on(goal,platform).
platform.
ramp.}).
%%Value was:2.215447154471546
#pos(a4,
{},
{},
{goal(a).
initiate(interact(a)).
on(goal,platform).
platform.
ramp.}).
%%Value was:-0.6113883847549909
#pos(a5,
{},
{},
{goal(a).
initiate(observe).
on(goal,platform).
platform.
ramp.}).
%%Value was:-0.6122076023391817
#pos(a6,
{},
{},
{goal(a).
initiate(rotate).
on(goal,platform).
platform.
ramp.}).
%%Value was:-0.6133705475810743
#pos(a7,
{},
{},
{goal(a).
initiate(balance).
on(agent,platform).
on(goal,platform).
platform.}).
%%Value was:15.921052631578945
#pos(a8,
{},
{},
{goal(a).
initiate(interact(a)).
on(agent,platform).
on(goal,platform).
platform.}).
%%V

In [ ]:
[i[1][0] for i in traces if 'moving' in i[1][0]]

In [ ]:
'visible' in traces[1][1]

In [ ]:
[i[0] for i in traces if "initiate(drop(left))" in i[0]]

In [ ]:
reward

In [ ]:
15/2